Load dataset:

In [28]:
from HumanGenomeDataset.load_dataset import load_dataset

data_df = load_dataset('dna_protein_coding_sequences')
data_df.head(2)

,ID,sequence,category
0,NM_001368254.1,GCTGAGCTGAGCTGGGGCGCAGCCGCCTGTCTGCACCGGCAGCACC...,NM
1,NM_001350977.1,TCCACAACTGAAACATCCACTTCTGAACACCATGTCCTACTACAGC...,NM


Start simulation:

In [29]:
from src.simulation import ProteinSinthesisProcess
SIM_TIME = 100

In [30]:
protein_synthesis_process = ProteinSinthesisProcess(dna_sequences_df=data_df, verbose=False)
protein_synthesis_process.run(simulation_time=SIM_TIME) # run the simulation

Results:

In [31]:
results_df = protein_synthesis_process.dna_sequences_df

In [32]:
print('Number of DNA sequence traslated:', results_df[results_df['mrna_sequences'].notna()].shape[0])
print('Number of DNA sequence not traslated:', results_df[results_df['protein_synthesized']==False].shape[0])

Number of DNA sequence traslated: 11
Number of DNA sequence not traslated: 7


In [33]:
results_df[results_df['mrna_sequences'].notna()][[
    'polypeptides_chains', 'number_of_proteins_synthesized']]

,polypeptides_chains,number_of_proteins_synthesized
977,[NH2-TR-COOH],1
14753,"[NH2-VNFAYFNIFSKPN-COOH, NH2-RLSTNSQFPIRLI-COO...",4
37085,[NH2--COOH],1
38699,"[NH2-W-COOH, NH2-GIIKQFT-COOH]",2
53884,"[NH2-QTNKTSYIKNMVGYYK-COOH, NH2-WV-COOH, NH2-Y...",5
76956,"[NH2--COOH, NH2-SLYSSRGLGNKHPLNMN-COOH, NH2--C...",9
92656,"[NH2--COOH, NH2-DMIC-COOH]",2
103731,[NH2--COOH],1
111094,"[NH2--COOH, NH2-VFGVYEFRFQPEHLEHISLSTG-COOH, N...",4
124961,"[NH2-FTC-COOH, NH2-NLYFKWRRPRHHPRV-COOH, NH2-L...",3
